In [13]:
import os
os.chdir('/nfs/homedirs/rachwan/Torch-Pruning')
import torch
import numpy as np

In [14]:
import mlflow
os.chdir('/nfs/homedirs/rachwan/Torch-Pruning/examples/yolo_example/yolov5')
from models.yolo import Model

In [3]:
model = Model('models/yolov5s.yaml', ch=3, nc=80, anchors=3).to('cuda')  # create

Overriding model.yaml anchors with anchors=3

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]      

In [4]:
os.chdir('/nfs/homedirs/rachwan/Torch-Pruning')
mlflow.set_tracking_uri("sqlite:///mlstore.db")

In [5]:
reg_model_name = "yolov5lpruned"

In [6]:
mlflow.sklearn.log_model(model, "yolov5",
                             registered_model_name=reg_model_name)

Context impl SQLiteImpl.
Will assume non-transactional DDL.
2023/01/11 12:10:42 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/01/11 12:10:50 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.13.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torchvision==0.13.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
/nfs/homedirs/rachwan/miniconda3/envs/structure_pruning/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'yolov5lpruned' already exists. Creating a new version of this model...
2023/01/11 12:10:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creati

In [7]:
model_uri = f"models:/{reg_model_name}/1"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [8]:
loaded_model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
 

In [3]:
from torch import package

In [4]:
imp = package.PackageImporter('/nfs/homedirs/rachwan/Torch-Pruning/yolov5lpruned.pt')
model = imp.load_pickle('yolov5lpruned', 'model.pkl')

In [12]:
model
params = sum([np.prod(p.size()) for p in model.parameters()])
print("Number of Parameters: %.1fM" % (params / 1e6))

Number of Parameters: 7.2M


In [9]:
os.chdir('/nfs/homedirs/rachwan/Torch-Pruning/examples/yolo_example/yolov5')


In [11]:
from evaluation_tool.models.metrics.Evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_inference(loaded_model, tuple([3, 640, 640]), 'cuda', iterations=100, batch_size=1)

evaluator.get_all_metrics()

{'time': 20.14771484375,
 'gpu': 0.45268326400000003,
 'disk': 953.981448,
 'cost': 4589.54,
 'emission': 0.03220131522483286,
 'energy': 0.10698111370376368}